In [3]:
from datetime import datetime, timedelta, timezone
import requests
import logging
import urllib3
import sys
import os
sys.path.append(os.path.abspath('../vault'))
from methods import VaultMethods

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%m/%d/%Y %I:%M:%S %p',
                    level=logging.INFO)

vault_session = VaultMethods()
vault_session.authenticate()
gravwell_token = vault_session.retrieve_gravwell_secret()

if gravwell_token:
    logging.info("Retrieved SIEM token")
else:
    logging.error("Failed to retrieve SIEM token")

headers = {
             "Gravwell-Token": gravwell_token, 
             "Content-Type": "application/json"
          }

def convert_datetime(datetime_str):
    dt = datetime.fromisoformat(datetime_str)
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")

def execute_direct_query(headers, siem: str, query: str) -> None:
    timestamp = datetime.now(timezone.utc) - timedelta(minutes=600)
    search_start = timestamp.isoformat()
    search_start = convert_datetime(search_start)
    search_conclude = datetime.now(timezone.utc).isoformat()
    search_start = convert_datetime(search_start)
    search_conclude = convert_datetime(search_conclude)
    payload = {"Background": True,
               "Name": "Direct Query",
               "NoHistory": True,
               "Preview": False,
               "Format": "text",
               "SearchEnd": search_conclude,
               "SearchStart": search_start,
               "SearchString": query}
    try:
        response = requests.post(headers=headers,
                                 url=f"https://{siem}/api/search/direct",
                                 json=payload,
                                 verify=False)
        response_content = response.content
        response_content = response_content.decode('utf-8')
        logging.info(response_content)
    except Exception as e:
        logging.exception("Error executing direct query: {0}".format(e))

siem = input("Enter the SIEM FQDN -> ")
query = "tag=_alerts"
execute_direct_query(headers, siem, query)

Enter the Vault FQDN->  vault.internal.subterfuge.biz
Enter the Vault username->  robert.gaines
Enter the Vault password->  ········


07/07/2025 10:49:14 PM Retrieved SIEM token


Enter the SIEM FQDN ->  siem.internal.subterfuge.biz


07/07/2025 10:49:30 PM {"Type":"event","Metadata":{"UID":1,"Username":"robert.gaines","Created":"2025-07-07T20:00:01.449030011Z","AlertID":"63895bae-c9d8-4738-9f25-88688bc46f8a","AlertName":"Fortigate VPN Logon","AlertActivation":"84f1ff96-65bd-447d-8a24-a0090e39a17d","EventIndex":0,"TargetTag":"_alerts","AlertLabels":["High","Fortigate","kit/io.gravwell.syslog"],"Dispatcher":{"Type":"scheduledsearch","ID":"1f1a3f28-649e-4735-8f13-0d50812dc351","Name":"Fortigate VPN Logon","SearchID":"43460738405","Labels":["Firewall","VPN","High","kit/io.gravwell.syslog"],"EventCount":1,"EventsElided":false},"Consumers":[{"Type":"flow","ID":"b4add7c8-60c2-46a7-8fdd-357de14c4d96","Name":" Fortigate VPN Logon - Flow","Labels":["High","Fortigate","kit/io.gravwell.syslog"]}],"UserMetadata":{}},"Contents":{"action":"tunnel-up","group":"N/A","logdesc":"IPsec connection status changed","remip":"172.56.203.77","subtype":"vpn","time":"12:58:07","tunneltype":"ipsec","user":"172.20.10.3"}}

